## Introduction
En este *Notebook* aprenderemos cómo manejar archivos DICOM en Python.<br />
Para cumplir esta tarea utilizamos un subconjunto de imágenes de TC del *Cancer Imaging Archive TCIA* (1, 2)<br />(https://www.kaggle.com/kmader/siim-medical-images) que contiene cortes únicos de TC.<br />

Si desea descargar los datos, inicie sesión en kaggle y navegue hasta esta página: https://www.kaggle.com/kmader/siim-medical-images

(1) Albertina, B., Watson, M., Holback, C., Jarosz, R., Kirk, S., Lee, Y., … Lemmerman, J. (2016). Radiology Data from The Cancer Genome Atlas Lung Adenocarcinoma [TCGA-LUAD] collection. The Cancer Imaging Archive. http://doi.org/10.7937/K9/TCIA.2016.JGNIHEP5

(2) Clark K, Vendt B, Smith K, Freymann J, Kirby J, Koppel P, Moore S, Phillips S, Maffitt D, Pringle M, Tarbox L, Prior F. The Cancer Imaging Archive (TCIA): Maintaining and Operating a Public Information Repository, Journal of Digital Imaging, Volume 26, Number 6, December, 2013, pp 1045-1057. (paper)

## Importaciones
* **pathlib** para un fácil manejo de rutas
* **pydicom** para manejar archivos dicom
* **matplotlib** para visualización
* **numpy** para crear el contenedor 3D

In [ ]:
from pathlib import Path

import pydicom 
import matplotlib.pyplot as plt
import numpy as np

## Lectura del archivo DICOM
Para esto, usamos la función ``read_file(path)`` de la librería **pydicom**

In [ ]:
sample_dcm = "ID_0000_AGE_0060_CONTRAST_1_CT.dcm"
dicom_file = pydicom.read_file(sample_dcm)

Echemos un vistazo a lo que contiene este archivo.
Puede imprimir el archivo DICOM para obtener una gran cantidad de información, que contiene, por ejemplo, la empresa que construyó el escáner (SIEMENS en este caso), la forma de la imagen (filas, columnas, 512x512 en este caso), la altura de la tabla, toda la información sobre el paciente (por supuesto, la información personal se anonimiza aquí) y, de gran importancia, la orientación de la imagen.

No dudes en desplazarte por la información disponible.le.

In [ ]:
print(dicom_file)

Puede acceder a las etiquetas dicom utilizando los identificadores codificados en hexadecimal al comienzo de cada línea. Como ejemplo, si desea obtener la forma de la imagen, puede utilizar esos dos identificadores.

* (0028, 0010) Rows
* (0028, 0011) Columns
* (0018, 0015) Body Part Examined

El 0x delante del identificador le dice al intérprete de Python que debe interpretar este valor como hexadecimal.

In [ ]:
print(dicom_file[0x0028, 0x0010])
print(dicom_file[0x0028, 0x0011])
print(dicom_file[0x0018, 0x0015])

Existe una forma alternativa y más directa de acceder a los valores de las etiquetas del encabezado DICOM utilizando las descripciones de las etiquetas.
Tenga en cuenta la inscripción: "Body Part Examined" se convierte en "'BodyPartExamined" (el llamado caso Pascal):

In [ ]:
print('Rows: ', dicom_file.Rows)
print('Columns: ', dicom_file.Columns)
print('Body Part Examined: ', dicom_file.BodyPartExamined)

## Accediendo a la información del cuerpo DICOM (la imagen real):

``pixel_array`` contiene los datos de la imagen real:

In [ ]:
ct = dicom_file.pixel_array
plt.figure()
plt.imshow(ct, cmap="cividis")

Podemos realizar una verificación rápida de cordura y asegurarnos de que la forma de la imagen corresponda a las Filas y Columnas que vimos anteriormente en la información del encabezado (512x512).

In [ ]:
print(ct.shape)

## Datos 3D
En esta sección veremos una resonancia magnética de cabeza completa, para que aprendas cómo manejar datos 3D almacenados como múltiples archivos DICOM 2D.

Los datos se toman de aquí (https://zenodo.org/record/16956#.YFMM5PtKiV5) (3)

Puede descargarlo directamente desde el enlace haciendo clic en el botón de descarga junto al botón de vista previa.

Nuevamente descomprimimos el directorio tras obtenerlo.

Normalmente hay un archivo para cada segmento, por lo que necesitamos leer todos los archivos y agregar los segmentos a una lista.

In [ ]:
path_to_head_mri = Path("SE000001")

Podemos usar la función ``glob`` para devolver todos los elementos de un directorio que correspondan al patrón proporcionado.
Como en este caso, el directorio solo contiene los archivos DICOM, podemos devolver todos los archivos que contiene ("*")

In [ ]:
all_files = list(path_to_head_mri.glob("*"))

In [ ]:
all_files

Ahora leeremos estos archivos usando el método ``read_file`` y los agregaremos a una lista.

In [ ]:
mri_data = []

for path in all_files:
    data = pydicom.read_file(path) # read the single DICOM files
    mri_data.append(data)
print(len(mri_data))

Como puede ver en las rutas impresas anteriores, es posible que los archivos DICOM no estén ordenados según su posición real de imagen. <br />
Esto se puede verificar inspeccionando ``SliceLocation``

In [ ]:
for slice in mri_data[:5]:
    print(slice.SliceLocation)

Es fundamental ordenarlos, ya que de lo contrario el escaneo completo se mezclaría y sería inútil.

Podemos usar el atributo ``SliceLocation`` pasado a la función ``sorted`` para identificar la posición del corte 2D y así ordenar los cortes.

In [ ]:
mri_data_ordered = sorted(mri_data, key=lambda slice: slice.SliceLocation) 

for slice in mri_data_ordered[:5]:
    print(slice.SliceLocation)

Ahora extraemos los datos reales (``pixel_arrays``) de los archivos DICOM y los almacenamos en una lista.

In [ ]:
full_volume = []
for slice in mri_data_ordered:
    full_volume.append(slice.pixel_array)
full_volume = np.array(full_volume)
print(full_volume.shape)

Y ahora podemos echar un vistazo a algunas porciones del volumen 3D ordenado:

In [ ]:
fig, axis = plt.subplots(3, 3, figsize=(10, 10))

slice_counter = 0
for i in range(3):
    for j in range(3):
        axis[i][j].imshow(full_volume[slice_counter], cmap="gray")
        slice_counter+=1

**¡Impresionante!** <br />
Ahora tenemos una forma de manejar datos 2D y 3D almacenados en formato DICOM.

Pero (como habrás notado), la lectura y ordenación manual de archivos parece un poco tediosa, sería genial si hubiera una herramienta que se encargara de esto por nosotros.

Lo existe y su nombre es SimpleITK https://pypi.org/project/SimpleITK/

SimpleITK proporciona funcionalidad para detectar y leer automáticamente todos los archivos dicom sin que usted administre la lectura del archivo o el orden de los cortes.

La rutina general es siempre idéntica.

1. Obtenga los ID de serie de todos los archivos del directorio. Esto es importante ya que también puede haber varios escaneos en el mismo directorio y no queremos mezclarlos. ``ImageSeriesReader.GetGDCMSeriesIDs(ruta)`` maneja esto y devuelve todos los ID que puede encontrar
2. Luego devolvemos todos los nombres de archivos en el directorio que tienen nuestro ID deseado ``ImageSeriesReader.GetGDCMSeriesFileNames(ruta, ID)`` proporciona esta funcionalidad
3. Luego definimos el lector de imágenes llamado ``ImageSeriesReader()`` y le damos los nombres de los archivos usando ``SetFileNames(file_names)``
4. Finalmente ejecutamos el lector para obtener los datos deseados llamando a ``Execute()``

Vamos:
Al principio importamos las bibliotecas necesarias:

In [ ]:
import SimpleITK as sitk

In [ ]:
series_ids = sitk.ImageSeriesReader.GetGDCMSeriesIDs(str(path_to_head_mri))
print(series_ids)

In [ ]:
series_file_names = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(str(path_to_head_mri), series_ids[0])
series_file_names  # Notice how the files are already ordered

In [ ]:
series_reader = sitk.ImageSeriesReader()
series_reader.SetFileNames(series_file_names)

In [ ]:
image_data = series_reader.Execute()

In [ ]:
print(image_data.GetSize())

### Esto es todo lo que tienes que hacer para obtener tus datos volumétricos completos.
Como puede ver, la forma es (256, 256, 27), mientras que arriba la forma era (27, 256, 256).
Esto se debe simplemente a un orden diferente de dimensiones de la imagen. <br />

El último paso que debemos realizar es la conversión del objeto de imagen sitk en una matriz numpy. Esto se puede hacer llamando a ``GetArrayFromImage(image_data)``

In [ ]:
head_mri = sitk.GetArrayFromImage(image_data)
print(type(head_mri))
print(head_mri.shape)

Como puede ver, también movió directamente el canal de corte al frente. ¡Genial!
¡Ahora podemos echar un vistazo a nuestras imágenes y como puedes ver el resultado es idéntico al de arriba!

In [ ]:
fig, axis = plt.subplots(3, 3, figsize=(10, 10))

slice_counter = 0
for i in range(3):
    for j in range(3):
        axis[i][j].imshow(head_mri[slice_counter], cmap="gray")
        slice_counter+=1

Ahora tienes todas las herramientas para trabajar con archivos almacenados en formato DICOM.